# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/wca-ipy/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Generic Class

Generic class to ensure that all custom classes are printable

In [2]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Formatting Functions

Functions to convert results to and from seconds

In [3]:
def numSeconds(value):
    '''Convert float or string to number of seconds - e.g. 1:05.31 returns 65.31'''   
    
    if isinstance(value, float):
        return round(value, 2)
    elif ':' in value:
        parts = value.split(':')
        return round(int(parts[0]) * 60 + float(parts[1]), 2)
    else:
        return None


def formatResult(value):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value:
        if value > 60:
            return '{:d}:{:05.2f}'.format(int(value // 60), value - int(value // 60) * 60)
        else:
            return '{:.2f}'.format(value)

## Competition Classes

All of the classes relating to competition results

In [4]:
class Result(Printable):
    def __init__(self, date, rank, name, age, result, best, average, awards, solves, link):
        self.date, self.rank = date, rank
        self.name, self.age = name, age
        self.result, self.best, self.average = result, best, average
        self.awards = awards
        self.solves, self.link = solves, link

In [5]:
class Event(Printable):
    def __init__(self, name):
        self.name = name
        self.results = []

In [6]:
class Person(Printable):
    def __init__(self, name):
        self.name = name
        self.bestSingles = {}
        self.bestAverages = {}
        self.events = {}

In [7]:
from xlrd import open_workbook
from datetime import datetime

import Levenshtein

class Competition(Printable):
    def __init__(self, xslx):
        self.xlsx = xslx
        self.date = os.path.basename(os.path.dirname(xslx))
        self.events = []
        
        
    def saveEvent(self, eventName, resultName, results):
        '''Save results as markdown'''

        docsDir = os.path.join(projdir, 'docs', self.date)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, eventName + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## Senior Cubers Worldwide\n')
            f.write('### {} Competition {}\n\n'.format(eventName, self.date))
            
            f.write('| # | Name | Age | {} | Awards |'.format(resultName))
            for i in range(len(results[0].solves)):
                f.write('Solve {} |'.format(i + 1))
            f.write('Video link |\n')

            f.write('| :--: | -- | :--: | --: | :--: |')
            for i in results[0].solves:
                f.write(' --: |')
            f.write(' :-- |\n')

            for result in results:
                f.write('| {:d} | {} | {} |'.format(result.rank, result.name, result.age))
                if result.result:
                    f.write('{} |'.format(formatResult(result.result)))
                else:
                    f.write('DNF |')
                f.write('{} |'.format(result.awards))
                for solve in result.solves:
                    if solve:
                        f.write('{} |'.format(formatResult(solve)))
                    else:
                        f.write('DNF |')
                if 'http' in result.link:
                    f.write('[Link]({}) |'.format(result.link))
                else:
                    f.write('|')
                f.write('\n')

        
    def processSheets(self):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        wb = open_workbook(self.xlsx)

        for sheet in wb.sheets():
            eventName = sheet.name
            print('Processing {} for {}...'.format(eventName, self.date))

            # Fixed columns
            indexRank = 0
            indexName = 1
            indexAge = 2
            indexResult = 3

            # Variable columns
            resultName = None
            labelAverage = None
            indexAverage = None
            indexAwards = None
            indexSolves = []
            indexLink = None
            
            # Counters, etc
            rowNo = 1
            prevResult = None
            results = []

            for row in range(sheet.nrows):
                values = []
                for col in range(sheet.ncols):
                    values.append(sheet.cell(row,col).value)

                # Process header row - must be row 3
                if rowNo == 3:
                    for i in range(len(values)):
                        if str(values[i]).startswith('Award'):
                            indexAwards = i
                        elif str(values[i]).startswith('Solve'):
                            indexSolves.append(i)
                        elif 'Mo3' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'Ao5' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'link' in str(values[i]).lower():
                            indexLink = i
                    resultName = values[indexResult]

                # Process result row - must be after header on row 3
                if rowNo > 3:
                    # Fixed fields
                    name = values[indexName]
                    age = values[indexAge]
                    
                    # Determine result in seconds and rank
                    result = numSeconds(values[indexResult])
                    if result != prevResult:
                        rank = rowNo - 3

                    # Sanity checks
                    if (result and prevResult and result < prevResult):
                        print('ERROR: Order is incorrect for', name)
                    if (values[indexRank] != rank):
                        print('ERROR: Rank is incorrect for', name)

                    # Process solves - determine best, average, etc.
                    solves = []
                    best = None
                    for indexSolve in indexSolves:
                        value = numSeconds(values[indexSolve])
                        if value:
                            if best is None or best > value:
                                best = value
                        solves.append(value)
                    average = numSeconds(values[indexAverage])
                    awards = values[indexAwards]
                    link = values[indexLink]
                    
                    # Create person
                    if name in persons:
                        person = persons[name]
                    else:
                        for person in persons:
                            if (Levenshtein.distance(person, name) < 5):
                                print("WARNING: Similar names -", person, "+", name)
                        person = Person(name)
                        persons[name] = person

                    # Update best single
                    if eventName in person.bestSingles:
                        if best and person.bestSingles[eventName] >= best:
                            person.bestSingles[eventName] = best
                            if '⚡' not in awards:
                                print('WARNING: Missing ⚡ for', name)
                        else:
                            if '⚡' in awards:
                                print('WARNING: Spurious ⚡ for', name)
                    else:
                        person.bestSingles[eventName] = best

                    # Update best average
                    if eventName in person.bestAverages:
                        if average and person.bestAverages[eventName] >= average:
                            person.bestAverages[eventName] = average
                            if '🔥' not in awards:
                                print('WARNING: Missing 🔥 for', name)
                        else:
                            if '🔥' in awards:
                                print('WARNING: Spurious 🔥 for', name)
                    else:
                        person.bestAverages[eventName] = average

                    # Record result
                    payload = Result(self.date, rank, name, age, result, best, average, awards, solves, link)
                    results.append(payload)
                    
                    if eventName in person.events:
                        person.events[eventName].results.insert(0, payload)
                    else:
                        event = Event(eventName)
                        event.results.append(payload)
                        person.events[eventName] = event

                    prevResult = result

                rowNo += 1

            event = Event(eventName)
            event.results = results
            self.events.append(event)
            
            self.saveEvent(eventName, resultName, results)
            print()

In [8]:
import glob

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
xlsxPattern = '*.xlsx'

persons = {}
competitions = {}

for xlsx in glob.glob(os.path.join(projdir, 'data', datePattern, xlsxPattern)):
    competition = Competition(xlsx)
    competition.processSheets()
    competitions[competition.date] = competition

outFile = os.path.join(projdir, 'docs', 'README.md')
with open(outFile, 'w') as f:
    f.write('## Senior Cubers Worldwide\n')
    f.write('### Weekly Competitions Archive\n')
    
    for competition in sorted(competitions.keys(), reverse = True):
        f.write('{} - '.format(competition))
        events = []
        for event in competitions[competition].events:
            events.append('[{}]({}/{}.md)'.format(event.name, competitions[competition].date, event.name))
        f.write('{}\n\n'.format(', '.join(events)))
        
    f.write('Python code can be found at [https://github.com/Logiqx/scw-comp](https://github.com/Logiqx/scw-comp)\n')

Processing 3x3x3 for 2020-02-03...

Processing 3x3x3 for 2020-02-10...

Processing 4x4x4 for 2020-02-10...

Processing 3x3x3 for 2020-02-17...

Processing 2x2x2 for 2020-02-17...

Processing 3BLD for 2020-02-17...



## All Done!